## Importer

In [1]:
import os
import zipfile
import hashlib
import shutil # Används för att säkert flytta filer
from pathlib import Path
from collections import Counter

In [2]:
TEST_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\00_test")
REAL_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw")

# *** Sätt vilken mapp skriptet ska köras på HÄR: ***
# ---------------------------------------------------
#TARGET_DIR = TEST_DIR  # <-- BÖRJA ALLTID MED TEST_DIR
TARGET_DIR = REAL_DIR # <-- Använd denna FÖRST EFTER TEST
# ---------------------------------------------------

if not TARGET_DIR.exists():
    print(f"FEL: Mappen {TARGET_DIR} finns inte.")
    print("Skapa den och kopiera dit test-filer innan du fortsätter.")
else:
    print(f"SKRIPTET ÄR INSTÄLLT ATT KÖRA PÅ:")
    print(f"==> {TARGET_DIR}")
    if "00_test" not in str(TARGET_DIR):
        print("\n*** VARNING! DU KÖR PÅ DIN RIKTIGA DATA! ***")
        print("Se till att du har testat på '00_test' först.")

SKRIPTET ÄR INSTÄLLT ATT KÖRA PÅ:
==> C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw

*** VARNING! DU KÖR PÅ DIN RIKTIGA DATA! ***
Se till att du har testat på '00_test' först.


## Filinventering

In [3]:
def run_file_inventory(directory: Path):
    """Söker igenom och räknar filtyper."""
    print(f"\n--- Startar inventering av: {directory} ---")
    file_extensions = []
    for file_path in directory.rglob('*'):
        if file_path.is_file():
            extension = file_path.suffix.lower()
            file_extensions.append(extension if extension else "<ingen filändelse>")

    file_counts = Counter(file_extensions)
    total_files = 0
    
    if not file_counts:
        print("Mappen är tom eller innehåller inga filer.")
        return

    print("Resultat:")
    for ext, count in file_counts.most_common():
        print(f"{ext:<15} {count} filer")
        total_files += count
    print("-" * 30)
    print(f"{'TOTALT':<15} {total_files} filer")

# Kör inventeringen FÖRE
run_file_inventory(TARGET_DIR)


--- Startar inventering av: C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw ---
Resultat:
.pdf            4534 filer
.xlsx           121 filer
.docx           7 filer
.html           1 filer
.eml            1 filer
.txt            1 filer
------------------------------
TOTALT          4665 filer


#### Hitta en specifik filtyp

In [4]:
from pathlib import Path

def find_and_log_files_by_extension(target_directory: Path, extension: str):
    """
    Söker rekursivt efter filer med en specifik filändelse och skriver ut deras sökvägar.

    Args:
        target_directory (Path): Mappen där sökningen ska börja.
        extension (str): Filändelsen att söka efter (t.ex. '.heic' eller 'pdf').
    """
    # Säkerställ att filändelsen börjar med en punkt
    if not extension.startswith('.'):
        extension = '.' + extension.lower()
    else:
        extension = extension.lower()
        
    print(f"\n--- Söker efter filer med ändelsen: {extension} i {target_directory.name} ---")
    
    found_files = list(target_directory.rglob(f'*{extension}'))
    
    if not found_files:
        print(f"Hittade inga filer med ändelsen {extension}.")
        return

    print(f"Hittade {len(found_files)} fil(er):")
    
    for i, file_path in enumerate(found_files, 1):
        print(f"  {i}. {file_path}")

# ***************************************************************
# ANVÄNDNING
# ***************************************************************

# Exempel 1: Hitta alla .heic filer (baserat på din inventering fanns 2 st)
find_and_log_files_by_extension(TARGET_DIR, '.heic')

# Exempel 2: Hitta alla .dwg filer (baserat på din inventering fanns 3 st)
find_and_log_files_by_extension(TARGET_DIR, 'dwg') # Du kan även skriva utan punkten

# Exempel 3: Hitta alla .mov filer
find_and_log_files_by_extension(TARGET_DIR, '.mov')


--- Söker efter filer med ändelsen: .heic i 01_raw ---
Hittade inga filer med ändelsen .heic.

--- Söker efter filer med ändelsen: .dwg i 01_raw ---
Hittade inga filer med ändelsen .dwg.

--- Söker efter filer med ändelsen: .mov i 01_raw ---
Hittade inga filer med ändelsen .mov.


### ZIP-filsuppackning och radering av orginalfiler

In [5]:
zip_files = list(TARGET_DIR.rglob('*.zip'))
print(f"Hittade {len(zip_files)} ZIP-filer att bearbeta.")

for zip_file_path in zip_files:
    print(f"\nBearbetar: {zip_file_path.name}...")
    
    # Vi packar upp till mappen där zip-filen ligger
    extract_folder = zip_file_path.parent
    
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"  -> Uppackad till: {extract_folder}")
        
        # Om uppackningen lyckades, RADERA zip-filen
        os.remove(zip_file_path)
        print(f"  -> Original .zip-fil raderad.")
        
    except zipfile.BadZipFile:
        print(f"  -> FEL: Kunde inte öppna {zip_file_path.name}. Filen kan vara korrupt. Raderas ej.")
    except Exception as e:
        print(f"  -> FEL: Ett okänt fel uppstod: {e}. Raderas ej.")

print("\nZIP-bearbetning klar.")

Hittade 0 ZIP-filer att bearbeta.

ZIP-bearbetning klar.


### Hitta och radera dubbletter (In-Place) och flytta oanvändbara filer till en egen mapp 01_raw_unsupported

In [6]:
# *********************************************************************************
# 1. DEFINITIONER (Behövs i varje cell om de inte kommer från importer)
# *********************************************************************************
# Listan med de KÄRNSYMBOLER som SKA SKYDDAS.
# Denna lista måste finnas tillgänglig för remove_empty_landscape_folders().
CORE_LANDSCAPE_NAMES = [
    'blekinge', 'dalarna', 'gotland', 'gävleborg', 'halland', 'jämtland', 
    'jönköping', 'kalmar', 'kronoberg', 'norrbotten', 'skåne', 'stockholm', 
    'södermanland', 'uppsala', 'värmland', 'västerbotten', 'västernorrland', 
    'västmanland', 'västra götaland', 'örebro', 'östergötland'
]


# *********************************************************************************
# 2. FUNKTIONER (Behövs i denna cell)
# *********************************************************************************

# Går igenom alla filer i mappen. Beräknar en hash för innehållet.
def get_file_hash(file_path: Path) -> str:
    """Beräknar SHA256-hashen för en fil."""
    sha256 = hashlib.sha256()
    try:
        with open(file_path, 'rb') as f:
            while chunk := f.read(8192):
                sha256.update(chunk)
        return sha256.hexdigest()
    except IOError:
        return "" # Vid läsfel


def is_protected_folder(folder_name: str, core_names: list[str]) -> bool:
    """Kollar om mappnamnet innehåller något av kärnsymbolerna i listan."""
    folder_name_lower = folder_name.lower()
    for name in core_names:
        if name in folder_name_lower:
            return True
    return False


def remove_empty_landscape_folders(target_dir: Path, core_protected_names: list[str]):
    """Raderar tomma mappar i target_dir, men hoppar över skyddade mappar."""
    removed_count = 0
    
    # Går igenom alla mappar från djupast till grundast (inifrån och ut)
    for folder_path in sorted(target_dir.rglob('*'), reverse=True):
        if folder_path.is_dir():
            
            # Kontroll 1: Skydda Landskapsmappar (flexibel matchning)
            if is_protected_folder(folder_path.name, core_protected_names):
                continue
            
            # Kontroll 2: Kontrollera om mappen är tom
            try:
                if not any(folder_path.iterdir()):
                    os.rmdir(folder_path)
                    removed_count += 1
                    print(f"  [Rensat] Raderade tom mapp: {folder_path.relative_to(target_dir)}")
            except OSError as e:
                print(f"  [FEL] Kunde inte radera {folder_path.relative_to(target_dir)}: {e}")

    print(f"\nTotalt antal tomma mappar rensade: {removed_count}")


# *********************************************************************************
# 3. HUVUDPROCESS
# *********************************************************************************
seen_hashes = {} 
duplicate_count = 0
file_count = 0

print("Startar dubblettkontroll (In-Place)... Detta kan ta tid.")

# Steg 1: Dubblettkontroll och radering av filer
for file_path in TARGET_DIR.rglob('*'):
    if not file_path.is_file():
        continue
    
    if file_path.suffix.lower() == '.zip':
        continue
    
    file_count += 1
    if file_count % 1000 == 0:
        print(f"  ... har kontrollerat {file_count} filer ...")

    file_hash = get_file_hash(file_path)
    
    if not file_hash:
        continue

    if file_hash in seen_hashes:
        duplicate_count += 1
        # RADERA DUBBLETTFILEN OMEDELBART
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"  -> FEL: Kunde inte radera dubblettfilen: {file_path.name}: {e}")
    else:
        seen_hashes[file_hash] = file_path


print("\n--- Dubblettkontroll klar! ---")
print(f"Totalt antal unika filer (original) som behållits: {len(seen_hashes)}")
print(f"Totalt antal dubbletter hittade och raderade: {duplicate_count}")


# Steg 2: Rensa Tomma Mappar (Återställt från ditt ursprungliga inlägg)
print("\nStartar rensning av tomma mappar...")
remove_empty_landscape_folders(TARGET_DIR, CORE_LANDSCAPE_NAMES)


# Steg 3: Flytta Filer som inte stöds (Korrigerat för att köra korrekt)
# BASE_DIR och DATA_DIR är definierade i en tidigare cell
DATA_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data")
UNSUPPORTED_DIR = DATA_DIR / "01_raw_unsupported"

# Skapa mappen om den inte redan finns
UNSUPPORTED_DIR.mkdir(parents=True, exist_ok=True)
print(f"\nMapp för filer som inte stöds finns/skapades på: {UNSUPPORTED_DIR}")

# Lista över filändelser som vi INTE kan bearbeta
unsupported_extensions = ['.jpg', '.heic', '.dwg', '.mov']

print(f"Genomsöker {TARGET_DIR} efter filer att flytta...")
    
files_to_move = []
for ext in unsupported_extensions:
    files_to_move.extend(TARGET_DIR.rglob(f'*{ext}'))

if not files_to_move:
    print("Hittade inga filer med filändelserna: ", unsupported_extensions)
else:
    print(f"Hittade {len(files_to_move)} filer att flytta...")
    
    moved_count = 0
    failed_count = 0
    
    for file_path in files_to_move:
        try:
            destination_name = f"{file_path.stem}_{file_path.parent.name}{file_path.suffix}"
            destination_path = UNSUPPORTED_DIR / destination_name
            
            counter = 1
            while destination_path.exists():
                destination_name = f"{file_path.stem}_{file_path.parent.name}_{counter}{file_path.suffix}"
                destination_path = UNSUPPORTED_DIR / destination_name
                counter += 1

            shutil.move(file_path, destination_path)
            print(f"  -> Flyttade: {file_path.name} TILL {destination_path.name}")
            moved_count += 1
            
        except Exception as e:
            print(f"  -> FEL: Kunde inte flytta {file_path.name}. Fel: {e}")
            failed_count += 1
    
    print("\n--- Flytt klar! ---")
    print(f"Totalt antal filer flyttade: {moved_count}")
    print(f"Totalt antal misslyckade flyttar: {failed_count}")


print("\n--- Rensnings- och flyttprocessen klar! ---")

Startar dubblettkontroll (In-Place)... Detta kan ta tid.
  ... har kontrollerat 1000 filer ...
  ... har kontrollerat 2000 filer ...
  ... har kontrollerat 3000 filer ...
  ... har kontrollerat 4000 filer ...

--- Dubblettkontroll klar! ---
Totalt antal unika filer (original) som behållits: 4633
Totalt antal dubbletter hittade och raderade: 32

Startar rensning av tomma mappar...

Totalt antal tomma mappar rensade: 0

Mapp för filer som inte stöds finns/skapades på: C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw_unsupported
Genomsöker C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw efter filer att flytta...
Hittade inga filer med filändelserna:  ['.jpg', '.heic', '.dwg', '.mov']

--- Rensnings- och flyttprocessen klar! ---


### Filinventering Efter städning

In [7]:
run_file_inventory(TARGET_DIR)
print("\nStädningsprocessen är klar!")


--- Startar inventering av: C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw ---
Resultat:
.pdf            4502 filer
.xlsx           121 filer
.docx           7 filer
.html           1 filer
.eml            1 filer
.txt            1 filer
------------------------------
TOTALT          4633 filer

Städningsprocessen är klar!
